In [1]:
import pickle
import IPython.display as ipd
# feature extractoring and preprocessing data
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path
import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy import signal
import noisereduce as nr
#Reports
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import pickle

In [2]:
def f_high(y,sr):
    from scipy import signal
    b,a = signal.butter(10, 2000/(sr/2), btype='highpass')
    yf = signal.lfilter(b,a,y)
    return yf

In [3]:
def reduce_noise(y, sr):
    reduced_noise = nr.reduce_noise(y=y, sr=sr, thresh_n_mult_nonstationary=2,stationary=False)
    return f_high(reduced_noise, sr)

In [4]:
def feature_extractor(sound_path):
    sr = 32000
    y, y_sr = librosa.load(sound_path, mono=True)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=y_sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    y_denoise = reduce_noise(y, y_sr)
    chroma_stft = librosa.feature.chroma_stft(y=y_denoise, sr=y_sr)
    rmse = librosa.feature.rms(y=y_denoise)
    spec_cent = librosa.feature.spectral_centroid(y=y_denoise, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y_denoise, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y_denoise, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y_denoise)
    mfcc = librosa.feature.mfcc(y=y_denoise, sr=sr)
    to_append += f' {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
    for e in mfcc:
        to_append += f' {np.mean(e)}'
    # print(to_append)
    file = open('demo.csv', 'a', newline='')
    writer = csv.writer(file)
    writer.writerow(to_append.split())
    file.close()

In [5]:
audio_path = Path('./mallar3/')

In [29]:
import os
from os import listdir
from os.path import isfile, join
files = [f for f in listdir(audio_path) if isfile(join(audio_path, f))]

In [30]:
files

['XC11495.mp3', 'XC23521.mp3', 'XC25611.mp3', 'XC138603.mp3', 'XC26968.mp3']

## 3_class SGD

### RFE

In [31]:
header = 'chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' de_chroma_stft de_rmse de_spectral_centroid de_spectral_bandwidth de_rolloff de_zero_crossing_rate'
for i in range(1, 21):
    header += f' de_mfcc{i}'
header = header.split()
file = open('demo.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(header)
file.close()

for file in files:
    feature_extractor(audio_path/file)

data = pd.read_csv('./demo.csv')

class_3_rfe_sgd = pickle.load(open('../models/3_class_rfe_sgd.pkl', 'rb'))

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(np.array(data[class_3_rfe_sgd.feature_names_in_], dtype = float)))
class_3_rfe_sgd.predict(X)

array(['mallar3', 'comrav', 'barswa', 'mallar3', 'eucdov'], dtype='<U7')

### Simple SGD

In [32]:
header = 'chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' de_chroma_stft de_rmse de_spectral_centroid de_spectral_bandwidth de_rolloff de_zero_crossing_rate'
for i in range(1, 21):
    header += f' de_mfcc{i}'
header = header.split()
file = open('demo.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(header)
file.close()

for file in files:
    feature_extractor(audio_path/file)

data = pd.read_csv('./demo.csv')

class_3_sgd = pickle.load(open('../models/3_class_SGDClassifier.pkl', 'rb'))

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(np.array(data[class_3_sgd.feature_names_in_], dtype = float)))
class_3_sgd.predict(X)

array(['mallar3', 'comrav', 'comrav', 'mallar3', 'eucdov'], dtype='<U7')

## 5_class SGD

In [9]:
header = 'chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' de_chroma_stft de_rmse de_spectral_centroid de_spectral_bandwidth de_rolloff de_zero_crossing_rate'
for i in range(1, 21):
    header += f' de_mfcc{i}'
header = header.split()
file = open('demo.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(header)
file.close()

for file in files:
    feature_extractor(audio_path/file)

data = pd.read_csv('./demo.csv')

class_5_rfe_sgd = pickle.load(open('../models/5_class_rfe_sgd.pkl', 'rb'))
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(np.array(data[class_5_rfe_sgd.feature_names_in_], dtype = float)))
class_5_rfe_sgd.predict(X)

array(['comrav', 'comrav', 'barswa', 'mallar3', 'eucdov'], dtype='<U7')

In [10]:
header = 'chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' de_chroma_stft de_rmse de_spectral_centroid de_spectral_bandwidth de_rolloff de_zero_crossing_rate'
for i in range(1, 21):
    header += f' de_mfcc{i}'
header = header.split()
file = open('demo.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(header)
file.close()

for file in files:
    feature_extractor(audio_path/file)

data = pd.read_csv('./demo.csv')

class_5_sgd = pickle.load(open('../models/5_class_SGDClassifier.pkl', 'rb'))

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(np.array(data[class_5_sgd.feature_names_in_], dtype = float)))
class_5_sgd.predict(X)

array(['mallar3', 'comrav', 'barswa', 'mallar3', 'eucdov'], dtype='<U7')

## 3_class Random Forest

In [33]:
header = 'chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' de_chroma_stft de_rmse de_spectral_centroid de_spectral_bandwidth de_rolloff de_zero_crossing_rate'
for i in range(1, 21):
    header += f' de_mfcc{i}'
header = header.split()
file = open('demo.csv', 'w', newline='')
writer = csv.writer(file)
writer.writerow(header)
file.close()

for file in files:
    feature_extractor(audio_path/file)

data = pd.read_csv('./demo.csv')

class_r_rf = pickle.load(open('../models/3_class_RandomForestClassifier.pkl', 'rb'))

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(np.array(data[class_r_rf.feature_names_in_], dtype = float)))
class_r_rf.predict(X)

array(['comrav', 'mallar3', 'comrav', 'mallar3', 'eucdov'], dtype=object)